In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
df = pd.read_csv("../processed_data/final_pre-processed.csv")

In [3]:
df = df.drop('Unnamed: 0',axis=1)[:4000]
df["Ratings"] = df["Ratings"].astype(int)

In [4]:
text_vectorizer = TfidfVectorizer(max_df=.8)
text_vectorizer.fit(df['reviewText'])
def rate(r):
    ary2 = []
    for rating in r:
        tv = [0,0,0,0,0]
        tv[rating-1] = 1
        ary2.append(tv)
    return np.array(ary2)

In [5]:
X = text_vectorizer.transform(df['reviewText']).toarray()
y = rate(df['Ratings'].values)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2)

In [6]:
global_model = Sequential()
global_model.add(Dense(128,input_dim=X_train.shape[1]))
global_model.add(Dense(5,activation='softmax'))

Metal device set to: Apple M1


In [7]:
global_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [8]:
NUM_ROUNDS = 10
NUM_CLIENTS = 10
BATCH_SIZE = 10
EPOCHS_PER_ROUND = 5
LEARNING_RATE = 0.1
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-7

In [9]:
x_train_clients = np.array_split(X_train,NUM_CLIENTS)
y_train_clients = np.array_split(y_train,NUM_CLIENTS)

x_test_clients = np.array_split(X_test,NUM_CLIENTS)
y_test_clients = np.array_split(y_test,NUM_CLIENTS)

In [20]:
client_models = []
for i in range(NUM_CLIENTS):
    local_model = Sequential()
    local_model.add(Dense(128,input_dim=X_train.shape[1]))
    local_model.add(Dense(5,activation='softmax'))
    local_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    local_model.fit(x_train_clients[i],y_train_clients[i],epochs=EPOCHS_PER_ROUND, batch_size=BATCH_SIZE,verbose=0)
    acc = local_model.evaluate(x_test_clients[i],y_test_clients[i])
    if acc[1] > 0.7:
        print(acc)
        client_models.append(local_model)

3/3 [==============================] - 0s 24ms/step - loss: 0.8276 - accuracy: 0.7375
[0.8275967836380005, 0.737500011920929]
3/3 [==============================] - 0s 25ms/step - loss: 0.8737 - accuracy: 0.7250
[0.8736509084701538, 0.7250000238418579]
3/3 [==============================] - 0s 19ms/step - loss: 0.7227 - accuracy: 0.7500
[0.7226929068565369, 0.75]
3/3 [==============================] - 0s 20ms/step - loss: 0.8350 - accuracy: 0.7500
[0.834956169128418, 0.75]
3/3 [==============================] - 0s 20ms/step - loss: 0.8131 - accuracy: 0.7375
[0.8131133913993835, 0.737500011920929]
3/3 [==============================] - 0s 19ms/step - loss: 0.8429 - accuracy: 0.7500
[0.842901885509491, 0.75]
3/3 [==============================] - 0s 21ms/step - loss: 0.9045 - accuracy: 0.6500


In [21]:
acc = global_model.evaluate(X_test,y_test)
print(acc)

25/25 [==============================] - 0s 8ms/step - loss: 19.6130 - accuracy: 0.6475
[19.613012313842773, 0.6474999785423279]


In [22]:
# SERVER ADAM OPTIMIZER 
fedadam = tf.optimizers.Adam(learning_rate=LEARNING_RATE,beta_1=BETA_1,beta_2 = BETA_2,epsilon=EPSILON)

In [23]:
# Performing Federated Averaging
weights = global_model.get_weights()
for i in range(len(weights)):
    for j in range(len(client_models)):
        client_weights = client_models[j].get_weights()
        weights[i] += client_weights[i]/NUM_CLIENTS

global_model.set_weights(weights)

In [24]:
acc = global_model.evaluate(X_test,y_test)
print(acc)

25/25 [==============================] - 0s 8ms/step - loss: 19.7477 - accuracy: 0.6463
[19.747737884521484, 0.6462500095367432]


In [25]:
# Train the Fed AVG model
fed_model = global_model
fed_model.compile(optimizer=fedadam, loss='categorical_crossentropy',metrics=['accuracy'])
fed_model.fit(X_train,y_train,epochs=5,batch_size=20)

Epoch 1/5
160/160 [==============================] - 2s 12ms/step - loss: 3.3049 - accuracy: 0.9003
Epoch 2/5
160/160 [==============================] - 2s 11ms/step - loss: 2.9774 - accuracy: 0.9134
Epoch 3/5
160/160 [==============================] - 2s 11ms/step - loss: 3.0336 - accuracy: 0.9200
Epoch 4/5
160/160 [==============================] - 2s 11ms/step - loss: 3.4111 - accuracy: 0.9172
Epoch 5/5
160/160 [==============================] - 2s 11ms/step - loss: 3.4630 - accuracy: 0.9253


In [26]:
acc = fed_model.evaluate(X_test,y_test)
print(acc)

25/25 [==============================] - 0s 9ms/step - loss: 48.0231 - accuracy: 0.6275
[48.023101806640625, 0.6274999976158142]
